In [20]:
from typing import Any, Dict, List
import random
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import  BaseExampleSelector

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

class RandomExampleSelector(BaseExampleSelector):
    
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example: Dict[str, str]) -> Any:
        pass

    def select_examples(self, input_variables):
        return random.choice(self.examples)


# example_prompt = ChatPromptTemplate.from_messages([
#     ("human", "What do you know about {country}?"),
#     ("ai", "{answer}"),
# ])

selector = RandomExampleSelector(
    examples=examples,
)

random_example = selector.select_examples({"country": "France"})

final_prompth = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert, you give short one line answer"),
    ("ai", random_example["answer"]),
    ("human", "What do you know about {country}?")
])

final_prompth.format(country="Brazil")

'System: You are a geography expert, you give short one line answer\nAI: \n        I know this:\n        Capital: Athens\n        Language: Greek\n        Food: Souvlaki and Feta Cheese\n        Currency: Euro\n        \nHuman: What do you know about Brazil?'